# Step 1: Install Langchain if you haven't

In [1]:
# !pip install langchain

In [3]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.llms.openai import OpenAI
from langchain.sql_database import SQLDatabase

# Step 2: Setup OPENAI 

In [1]:
%env OPENAI_API_KEY=sk-0SyUgp_open_ai_api_no_quotation_marks

env: OPENAI_API_KEY=sk-0SyUgp_open_ai_api_no_quotation_marks


In [4]:
ChinoodPATH="sqlite:///./resource/Chinook.db"
db = SQLDatabase.from_uri(ChinoodPATH)

In [9]:
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))



In [30]:
agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

# Step 3: Try to ask question about the Database
1. general description of the table.
2. retrive information with constraint.
3. sort table and retrive relevant information

In [31]:
agent_executor.run("Describe the playlisttrack table")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: The PlaylistTrack table is the most relevant to the question.
Action: sql_db_schema
Action Input: PlaylistTrack
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought: I now know the final answer
Final Answer: The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integers and form a primary key. It also has two foreign keys, one to the Track table and one to the Playlist table.

> Finished chain.


'The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integers and form a primary key. It also has two foreign keys, one to the Track table and one to the Playlist table.'

In [32]:
agent_executor.run("List ten artist whose last name start with letter 'B'")





> Entering new  chain...
Thought: I should use the sql_db_list_tables tool to get a list of tables in the database.
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: The Artist table looks like the most relevant table.
Action: sql_db_schema
Action Input: Artist
Observation: 
CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/
Thought: I should query the Artist table for artists whose last name start with 'B'.
Action: sql_db_query
Action Input: SELECT Name FROM Artist WHERE Name LIKE 'B%' LIMIT 10;
Observation: [('BackBeat',), ('Billy Cobham',), ('Black Label Society',), ('Black Sabbath',), ('Body Count',), ('Bruce Dickinson',), ('Buddy Guy',), ('Bebel Gilberto',), ('Baby Consuelo',), ('Banda Black Rio',)]
Thought: I now know the final an

'BackBeat, Billy Cobham, Black Label Society, Black Sabbath, Body Count, Bruce Dickinson, Buddy Guy, Bebel Gilberto, Baby Consuelo, Banda Black Rio'

In [34]:
agent_executor.run("Who are the top 3 best selling artists?")





> Entering new  chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: I should query the schema of the Album, Artist, and Invoice tables.
Action: sql_db_schema
Action Input: Album, Artist, Invoice
Observation: 
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Invoice" (
	"InvoiceId" INTEGER NOT NULL, 
	"CustomerId" INTEGER NOT NULL, 
	"InvoiceDate" DATETIME NOT NULL, 
	"Bi

'The top 3 best selling artists are Iron Maiden, U2, and Metallica.'